# Environment Setup

## Install Packages

In [ ]:
!pip install vllm triton -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 97.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.6/343.6 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 110.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 M

In [ ]:
!pip install -U bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 27.9 MB/s eta 0:00:00


## Imports

In [ ]:
import re
import gc
import torch
import pandas as pd

import triton
from vllm import LLM
from tqdm import tqdm
from vllm import SamplingParams

import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN') # Add your HuggingFace token (HF_TOKEN) to the environment variables

In [ ]:
# Load sample syllabi data
syllabi_data = pd.read_csv("https://raw.githubusercontent.com/LAiSER-Software/datasets/refs/heads/master/syllabi-data/preprocessed_50_opensyllabus_syllabi_data.csv")
syllabi_data.head()

,id,title,code,in_id,institution,in_country,in_state,in_city,in_latitute,in_longitude,in_description,description,learning_outcomes,year
0,4904852663176,Film Appreciation,COMM 2366,20005,South Plains College,United States,Texas,Levelland,33.576283,-102.367622,South Plains College (SPC) is a public communi...,"survey and analysis of cinema , including hist...",communications skills — to include effective w...,2022
1,661424964946,Quality Improvement Tools,PROD1252,1963,Niagara College,Canada,Ontario,Niagara Falls,43.079483,-79.090530,The Niagara College of Applied Arts and Techno...,you will be provided with a practical understa...,employ statistics to solve problems . use stat...,2022
2,6004364299427,"Data Privacy, Security, and Ethics",CSIS-2700,18664,Webster University,United States,Missouri,Webster Groves,38.592548,-90.357338,Webster University is a private university wit...,there is a subtle balance between improvements...,NaN,2022
3,8014409002470,Consumer Behavior,MKT 358,18014,Upper Iowa University,United States,Iowa,Fayette,42.839714,-91.797958,Upper Iowa University (UIU) is a private unive...,this course provides a survey of research find...,an overview of consumer behavior and terminolo...,2022
4,15728170259237,General Organic Chemistry I,CHM-235,20492,Chandler–Gilbert Community College,United States,Arizona,Chandler,33.295132,-111.797127,"public community college in Chandler, Arizona,...",rigorous introduction to chemistry of carbon -...,describe the bonding properties of the element...,2022


## Load Model with vLLM

In [ ]:
# Using quantization with bitsandbytes to reduce the model size and memory usage
llm = LLM(model="google/gemma-2-9b-it", dtype="bfloat16", quantization='bitsandbytes', load_format='bitsandbytes')

config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

INFO 01-07 17:21:43 config.py:510] This model supports multiple tasks: {'generate', 'score', 'classify', 'reward', 'embed'}. Defaulting to 'generate'.
WARNING 01-07 17:21:43 config.py:588] bitsandbytes quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 01-07 17:21:45 llm_engine.py:234] Initializing an LLM engine (v0.6.6.post1) with config: model='google/gemma-2-9b-it', speculative_config=None, tokenizer='google/gemma-2-9b-it', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=bitsandbytes, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=bitsandbytes, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='xgrammar'), observability_config=Obser

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 01-07 17:23:54 model_runner.py:1099] Loading model weights took 5.7563 GB
INFO 01-07 17:24:00 worker.py:241] Memory profiling takes 6.21 seconds
INFO 01-07 17:24:00 worker.py:241] the current vLLM instance can use total_gpu_memory (39.56GiB) x gpu_memory_utilization (0.90) = 35.61GiB
INFO 01-07 17:24:00 worker.py:241] model weights take 5.76GiB; non_torch_memory takes 0.28GiB; PyTorch activation peak memory takes 2.38GiB; the rest of the memory reserved for KV Cache is 27.18GiB.
INFO 01-07 17:24:01 gpu_executor.py:76] # GPU blocks: 5302, # CPU blocks: 780
INFO 01-07 17:24:01 gpu_executor.py:80] Maximum concurrency for 8192 tokens per request: 10.36x
INFO 01-07 17:24:03 model_runner.py:1415] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_utilizati

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:42<00:00,  1.20s/it]

INFO 01-07 17:24:46 model_runner.py:1535] Graph capturing finished in 42 secs, took 4.39 GiB
INFO 01-07 17:24:46 llm_engine.py:431] init engine (profile, create kv cache, warmup model) took 51.54 seconds


## Inference

In [ ]:
def vllm_batch_generate(queries, batch_size=32, key_skills=5):

    result = []

    sampling_params = SamplingParams(max_tokens=1000)

    # This a syllabi specific prompt template. Tweak the prompt for other input types.
    prompt_template = """<start_of_turn>user
**Objective:** Given a course syllabus description and its learning outcomes, complete the following tasks with structured outputs.

### Tasks:
1. **Skills Extraction:** Identify {num_key_skills} key skills mentioned in the description and learning outcomes.
   - Extract skill keywords or phrases of no more than three words.

2. **Skill Level Assignment:** Assign a proficiency level to each extracted skill based on the SCQF Level Descriptors (see below).

3. **Knowledge Required:** For each skill, list 3–5 broad areas of understanding or expertise necessary to develop the skill.

4. **Task Abilities:** For each skill, list 3–5 general tasks or capabilities enabled by the skill.

### Guidelines:
- **Skill Extraction:** Identify skills explicitly stated or implied through syllabus description and outcomes.
- **Skill Level Assignment:** Use the SCQF Level Descriptors to classify proficiency:
  - 1: Basic awareness of simple concepts.
  - 2: Limited operational understanding, guided application.
  - 3: Moderate knowledge, supervised application of techniques.
  - 4: Clear understanding, independent work in familiar contexts.
  - 5: Advanced knowledge, autonomous problem-solving.
  - 6: Specialized knowledge, critical analysis within defined areas.
  - 7: Advanced specialization, leadership in problem-solving.
  - 8: Expert knowledge, innovation in complex contexts.
  - 9: Highly specialized expertise, contributing original thought.
  - 10: Sustained mastery, influential in areas of specialization.
  - 11: Groundbreaking innovation, professional or academic mastery.
  - 12: Global expertise, leading advancements at the highest level.

- **Knowledge and Task Abilities:**
  - **Knowledge Required:** Broad areas, e.g., "data visualization techniques."
  - **Task Abilities:** General tasks or capabilities, e.g., "data analysis."
  - Each item in these two lists should be no more than three words.
  - Avoid overly specific or vague terms.

### Answer Format:
- Use this format strictly in the response:
  -> Skill: [Skill Name], Level: [1–12], Knowledge Required: [list], Task Abilities: [list].

### Input:
**Course Description:** {desc}

**Learning Outcomes:** {outcomes}

**Response:** Provide only the requested structured information without additional explanations.

<end_of_turn>
<start_of_turn>model
"""

    for i in range(0, len(queries), batch_size):
        prompts = [prompt_template.format(num_key_skills=key_skills, desc = queries.iloc[i]["description"], outcomes = queries.iloc[i]["learning_outcomes"]) for i in range(i, min(i+batch_size, len(queries)))]
        output = llm.generate(prompts, sampling_params=sampling_params)

        result.extend(output)

    return result

In [ ]:
result = vllm_batch_generate(syllabi_data, batch_size=50)

Processed prompts: 100%|██████████| 50/50 [00:23<00:00,  2.16it/s, est. speed input: 1618.94 toks/s, output: 418.89 toks/s]


In [ ]:
print(result[0].prompt)

<start_of_turn>user
**Objective:** Given a course syllabus description and its learning outcomes, complete the following tasks with structured outputs.

### Tasks:
1. **Skills Extraction:** Identify 5 key skills mentioned in the description and learning outcomes.
   - Extract skill keywords or phrases of no more than three words.

2. **Skill Level Assignment:** Assign a proficiency level to each extracted skill based on the SCQF Level Descriptors (see below).

3. **Knowledge Required:** For each skill, list 3–5 broad areas of understanding or expertise necessary to develop the skill.

4. **Task Abilities:** For each skill, list 3–5 general tasks or capabilities enabled by the skill.

### Guidelines:
- **Skill Extraction:** Identify skills explicitly stated or implied through syllabus description and outcomes.
- **Skill Level Assignment:** Use the SCQF Level Descriptors to classify proficiency:
  - 1: Basic awareness of simple concepts.
  - 2: Limited operational understanding, guided a

In [ ]:
print(result[0].outputs[0].text)

-> Skill: Film Analysis, Level: 3, Knowledge Required: film history, film techniques, societal impact, Task Abilities: analyze film, evaluate films, interpret narrative.
-> Skill: Communication, Level: 3, Knowledge Required: writing, speaking, visual media, Task Abilities: write responses, present ideas, create visuals.
-> Skill: Critical Thinking, Level: 3, Knowledge Required: analysis, evaluation, synthesis, Task Abilities: solve problems, form arguments, evaluate information.
-> Skill: Ethics, Level: 3, Knowledge Required: personal values, societal norms, decision-making, Task Abilities: make ethical choices, understand consequences, identify values.
-> Skill: Collaboration, Level: 3, Knowledge Required: teamwork, diverse roles, filmmaking process, Task Abilities: work in groups, understand contributions, appreciate teamwork. 





## Parse LLM Responses

In [ ]:
def parse_output(response):

    out = []
    items = [i.replace('\n', '') for i in response.strip('->').strip().split('->')]

    for item in items:
        skill_data = {}

        # Extract skill
        skill_match = re.search(r"Skill:\s*([^,]+)", item)
        if skill_match:
            skill_data['Skill'] = skill_match.group(1).strip()

        # Extract level
        level_match = re.search(r"Level:\s*(\d+)", item)
        if level_match:
            skill_data['Level'] = int(level_match.group(1).strip())

        # Extract knowledge required
        knowledge_match = re.search(r"Knowledge Required:\s*([^,]+(?:, [^:]+)*?)\s*(?:Task Abilities|$)", item)
        if knowledge_match:
            knowledge_raw = knowledge_match.group(1)
            skill_data['Knowledge Required'] = [k.strip() for k in knowledge_raw.strip().strip(',').split(',')]

        # Extract task abilities
        task_match = re.search(r"Task Abilities:\s*([^,]+(?:, [^:]+)*)", item)
        if task_match:
            task_raw = task_match.group(1)
            skill_data['Task Abilities'] = [t.strip() for t in task_raw.split(',')]

        out.append(skill_data)

    return out

In [ ]:
parsed_output = []
for i in range(len(result)):
    parsed = parse_output(result[i].outputs[0].text)
    for item in parsed:
        item['id'] = syllabi_data.iloc[i]['id']
        item['description'] = syllabi_data.iloc[i]['description']
        item['learning_outcomes'] = syllabi_data.iloc[i]['learning_outcomes']
    parsed_output.extend(parsed)

In [ ]:
len(syllabi_data), len(parsed_output)

(50, 247)

In [ ]:
parsed_output[0]

{'Skill': 'Film Analysis',
 'Level': 3,
 'Knowledge Required': ['film history', 'film techniques', 'societal impact'],
 'Task Abilities': ['analyze film', 'evaluate films', 'interpret narrative.'],
 'id': 4904852663176,
 'description': 'survey and analysis of cinema , including history , film techniques , production procedures , selected motion pictures , and cinema ’s impact on and reflection of society .',
 'learning_outcomes': 'communications skills — to include effective written , oral and visual communication . critical thinking skills — to include creative thinking , innovation , inquiry , and analysis , evaluation and synthesis of information . personal responsibility — to include the ability to connect choices , actions , and consequences to ethical decision - making . social responsibility — to include the demonstrated knowledge and competence , knowledge of civic responsibility , and the ability to engage effectively in regional , national , and global communities . analyze f

## Create DataFrame

In [ ]:
final_df = pd.DataFrame(parsed_output)
final_df = final_df[["id", "description", "learning_outcomes", "Skill", "Level", "Knowledge Required", "Task Abilities"]]
final_df

,id,description,learning_outcomes,Skill,Level,Knowledge Required,Task Abilities
0,4904852663176,"survey and analysis of cinema , including hist...",communications skills — to include effective w...,Film Analysis,3,"[film history, film techniques, societal impact]","[analyze film, evaluate films, interpret narra..."
1,4904852663176,"survey and analysis of cinema , including hist...",communications skills — to include effective w...,Communication,3,"[writing, speaking, visual media]","[write responses, present ideas, create visuals.]"
2,4904852663176,"survey and analysis of cinema , including hist...",communications skills — to include effective w...,Critical Thinking,3,"[analysis, evaluation, synthesis]","[solve problems, form arguments, evaluate info..."
3,4904852663176,"survey and analysis of cinema , including hist...",communications skills — to include effective w...,Ethics,3,"[personal values, societal norms, decision-mak...","[make ethical choices, understand consequences..."
4,4904852663176,"survey and analysis of cinema , including hist...",communications skills — to include effective w...,Collaboration,3,"[teamwork, diverse roles, filmmaking process]","[work in groups, understand contributions, app..."
...,...,...,...,...,...,...,...
242,1056561972620,this course is exciting because you will learn...,students will learn to interpret the language ...,Visual Interpretation,4,"[art history, visual language, stylistic analy...","[art analysis, visual critique, art description.]"
243,1056561972620,this course is exciting because you will learn...,students will learn to interpret the language ...,Writing,3,"[argumentation, grammar, style]","[essay writing, report writing, creative writi..."
244,1056561972620,this course is exciting because you will learn...,students will learn to interpret the language ...,Communication,3,"[public speaking, active listening, persuasive...","[class discussion, presentation delivery, coll..."
245,1056561972620,this course is exciting because you will learn...,students will learn to interpret the language ...,Art History,4,"[art movements, chronological understanding, c...","[art analysis, historical interpretation, them..."


# Taxonomy Crosswalk

In [ ]:
import spacy
import numpy as np
from tqdm import tqdm
from scipy.spatial.distance import cdist

import spacy.cli
spacy.cli.download("en_core_web_lg")

✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


Here we are only using ESCO data. But we can merge it with other taxonomies and run this code on the full data.

In [ ]:
esco_df = pd.read_csv("/content/ESCO skills Taxonomy.csv")
esco_df.head()

,conceptType,conceptUri,skillType,reuseLevel,preferredLabel,altLabels,hiddenLabels,status,modifiedDate,scopeNote,definition,inScheme,description
0,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/0005c151-5b5a...,skill/competence,sector-specific,manage musical staff,manage staff of music\ncoordinate duties of mu...,NaN,released,2023-11-30T15:53:37.136Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,Assign and manage staff tasks in areas such as...
1,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/00064735-8fad...,skill/competence,occupation-specific,supervise correctional procedures,oversee prison procedures\nmanage correctional...,NaN,released,2023-11-30T15:04:00.689Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Supervise the operations of a correctional fac...
2,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/000709ed-2be5...,skill/competence,sector-specific,apply anti-oppressive practices,apply non-oppressive practices\napply an anti-...,NaN,released,2023-11-28T10:45:53.54Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,"Identify oppression in societies, economies, c..."
3,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/0007bdc2-dd15...,skill/competence,sector-specific,control compliance of railway vehicles regulat...,monitoring of compliance with railway vehicles...,NaN,released,2023-11-30T16:29:18.273Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/skil...,"Inspect rolling stock, components and systems ..."
4,KnowledgeSkillCompetence,http://data.europa.eu/esco/skill/00090cc1-1f27...,skill/competence,cross-sector,identify available services,establish available services\ndetermine rehabi...,NaN,released,2023-11-28T10:38:49.206Z,NaN,NaN,http://data.europa.eu/esco/concept-scheme/memb...,Identify the different services available for ...


In [ ]:
def get_embedding(nlp, input_text):
    """
    Creates vector embeddings for input text based on nlp object

    Parameters
    ----------
    nlp : object of spacy nlp model
    input_text : text
        Provide text to be vectorized, usually skill, extracted of referenced

    Returns
    -------
    numpy array of vectorized text


    """
    doc = nlp(input_text)
    if len(doc) == 0:
        return np.zeros(300)  # Return zeros for empty texts
    return np.mean([word.vector for word in doc], axis=0)

nlp = spacy.load("en_core_web_lg")
esco_skill_embeddings = np.array([get_embedding(nlp, label) for label in esco_df['preferredLabel']])

In [ ]:
matches = []

raw_skill_embeddings = np.array([get_embedding(nlp, skill) for skill in final_df['Skill']])

# Calculate cosine similarities in bulk
similarities = 1 - cdist(raw_skill_embeddings, esco_skill_embeddings, metric='cosine')

for i, raw_skill in tqdm(enumerate(final_df['Skill'])):
    skill_matches = np.where(similarities[i] > 0.85)[0]
    for match in skill_matches:
        matches.append({
            "Research ID": final_df.iloc[i]['id'],
            "Description": final_df.iloc[i]['description'],
            "Learning Outcomes": final_df.iloc[i]['learning_outcomes'],
            "Raw Skill": raw_skill,
            "Level": final_df.iloc[i]['Level'],
            "Knowledge Required": final_df.iloc[i]['Knowledge Required'],
            "Task Abilities": final_df.iloc[i]['Task Abilities'],
            "Skill Tag": f"ESCO.{match}",
            "Correlation Coefficient": similarities[i, match]
        })

extracted = pd.DataFrame(columns=['Research ID', 'Description', 'Learning Outcomes', 'Raw Skill', 'Level', 'Knowledge Required', 'Task Abilities', 'Skill Tag', 'Correlation Coefficient'])
extracted = extracted._append(matches, ignore_index=True)

247it [00:00, 2526.90it/s]
<ipython-input-7-d47ec45c5ac5>:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  extracted = extracted._append(matches, ignore_index=True)


## Final Dataset

In [ ]:
extracted

,Research ID,Description,Learning Outcomes,Raw Skill,Level,Knowledge Required,Task Abilities,Skill Tag,Correlation Coefficient
0,4904852663176,"survey and analysis of cinema , including hist...",communications skills — to include effective w...,Communication,3,"[writing, speaking, visual media]","[write responses, present ideas, create visuals.]",ESCO.1170,0.863212
1,661424964946,you will be provided with a practical understa...,employ statistics to solve problems . use stat...,Data Analysis,4,"[data visualization, statistical methods, qual...","[quality monitoring, process improvement, tren...",ESCO.3226,0.907012
2,661424964946,you will be provided with a practical understa...,employ statistics to solve problems . use stat...,Data Analysis,4,"[data visualization, statistical methods, qual...","[quality monitoring, process improvement, tren...",ESCO.3637,0.925421
3,661424964946,you will be provided with a practical understa...,employ statistics to solve problems . use stat...,Data Analysis,4,"[data visualization, statistical methods, qual...","[quality monitoring, process improvement, tren...",ESCO.12451,0.878007
4,661424964946,you will be provided with a practical understa...,employ statistics to solve problems . use stat...,Communication,4,"[written expression, verbal skills, visual aids.]","[report writing, presentation delivery, inform...",ESCO.1170,0.863212
...,...,...,...,...,...,...,...,...,...
252,5952824680548,discusses personal and professional ethics wit...,apply self - assessment strategies to identify...,cultural intelligence,3,"[cultural diversity, intercultural communicati...","[adapt communication, build rapport, navigate ...",ESCO.11577,0.858077
253,11982958774840,an introductory course covering basic concepts...,NaN,Communication,3,"[technical writing, presentation skills, activ...","[report writing, class discussion, project exp...",ESCO.1170,0.863212
254,10462540337617,field - based learning opportunities in the bi...,NaN,Communication,3,"[scientific writing, oral presentation, interp...","[information sharing, report writing, group di...",ESCO.1170,0.863212
255,1056561972620,this course is exciting because you will learn...,students will learn to interpret the language ...,Visual Interpretation,4,"[art history, visual language, stylistic analy...","[art analysis, visual critique, art description.]",ESCO.1046,0.858496


## Save Datasets

In [ ]:
final_df.to_csv("final_df.csv", index = False)
extracted.to_csv("extracted.csv", index = False)